## Requirements

In [1]:
import numpy as np
import pandas as pd
import os, pickle, psutil

## Utils


In [2]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

# Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

## Load raw data

In [3]:
train_df = pd.read_csv('data/raw/sales_train_evaluation.csv')
prices_df = pd.read_csv('data/raw/sell_prices.csv')
calendar_df = pd.read_csv('data/raw/calendar.csv', parse_dates=['date'])

## Create Grid (train_df in long format)

#### Save original id order using index

In [ ]:
train_df.reset_index(inplace=True)

#### Complete train_df to include (unknown) evaluation sales

In [ ]:
train_df = train_df.reindex(columns=train_df.columns.tolist() + ['d_' + str(1942 + i) for i in range(28)])

#### Melt train_df

In [ ]:
grid_df = pd.melt(train_df, 
                  id_vars = ['index', 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                  var_name = 'd', 
                  value_name = 'sales')

print('Nb train rows:', len(train_df), 'to', len(grid_df))

del train_df

## Memory optimization

#### Grid

In [ ]:
print("{:>20}: {:>8}".format('Original grid_df', sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Convert categoricals
for col in ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']:
    grid_df[col] = grid_df[col].astype('category')

# Reduce numerics
grid_df['sales'] = grid_df['sales'].astype(np.float16)

print("{:>20}: {:>8}".format('Reduced grid_df', sizeof_fmt(grid_df.memory_usage(index=True).sum())))

#### Prices

In [ ]:
print("{:>20}: {:>8}".format('Original prices_df', sizeof_fmt(prices_df.memory_usage(index=True).sum())))

# Convert categoricals
for col in ['store_id', 'item_id']:
    prices_df[col] = prices_df[col].astype('category')
    
# Reduce numerics
prices_df['wm_yr_wk'] = prices_df['wm_yr_wk'].astype(np.int16)
prices_df['sell_price'] = prices_df['sell_price'].astype(np.float32) # not float16 for keeping the same precision

print("{:>20}: {:>8}".format('Reduced prices_df', sizeof_fmt(prices_df.memory_usage(index=True).sum())))

#### Calendar

In [ ]:
print("{:>20}: {:>8}".format('Original calendar_df', sizeof_fmt(calendar_df.memory_usage(index=True).sum())))

# Convert categoricals
cat_cols = ['weekday', 'event_name_1', 'event_type_1',
            'event_name_2', 'event_type_2','snap_CA',
            'snap_TX', 'snap_WI']
    
for col in cat_cols:
    calendar_df[col] = calendar_df[col].astype('category')

# Reduce numerics
calendar_df['wm_yr_wk'] = calendar_df['wm_yr_wk'].astype(np.int16)
calendar_df['wday'] = calendar_df['wday'].astype(np.int8)
calendar_df['month'] = calendar_df['month'].astype(np.int8)
calendar_df['year'] = calendar_df['year'].astype(np.int16)


print("{:>20}: {:>8}".format('Reduced calendar_df', sizeof_fmt(calendar_df.memory_usage(index=True).sum())))

## Merge all

In [ ]:
print('Original Grid size', grid_df.shape)

grid_df = merge_by_concat(grid_df, calendar_df, merge_on=['d'])
grid_df = merge_by_concat(grid_df, prices_df, merge_on=['store_id', 'item_id', 'wm_yr_wk'])

print('Final Grid size', grid_df.shape)

del calendar_df, prices_df

## Additionnal cleaning

#### Truncate TS before release date (the first filled sell price)

In [ ]:
print('Original Grid size', grid_df.shape)

grid_df = grid_df.dropna(subset=['sell_price']).reset_index(drop=True)

print('Final Grid size', grid_df.shape)

#### Convert 'd' to int


In [ ]:
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

#### Deal with NaN on categories

In [ ]:
#grid_df.isna().sum()

for col in ['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']:
    grid_df[col] = grid_df[col].cat.add_categories('None')
    grid_df[col].fillna('None', inplace=True)

#### Reorder

In [ ]:
grid_df = grid_df.sort_values(['index', 'd']).reset_index(drop=True)
grid_df.drop(['index'], axis=1, inplace=True)

## Save as interim data

In [ ]:
# Use pickle to not lose dtypes
grid_df.to_pickle('data/interim/grid_df.pkl')

grid_df.info()